# Policy refinement with human feedback (Policy v1 → v2)

이 노트북은 두 가지를 해준다:

1. **human_preferred_decision 컬럼이 포함된 피드백 테이블 템플릿 만들기**
   - 입력: `../results/disagreement_cases_with_text.csv`
   - 출력: `../results/disagreement_with_human_template.csv`
   - 여기에 사람이 `human_preferred_decision`을 채워 넣는다.

2. **사람 피드백 + 기존 정책 설명을 바탕으로, LLM에게 Policy A/B/C v2 초안을 요청**
   - 입력:
     - 사람이 라벨링한 `disagreement_with_human_filled.csv` (파일명은 아래에서 지정)
     - 기존 Policy A/B/C 설명 텍스트
   - 출력:
     - 터미널 출력 + `../results/policy_v2_suggestions.md` 파일


In [1]:
import os
from pathlib import Path
import pandas as pd
import textwrap

from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 디렉토리 구조 가정
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"

DISAGREE_CSV = RESULTS_DIR / "disagreement_cases_with_text.csv"

print("PROJECT_ROOT :", PROJECT_ROOT)
print("RESULTS_DIR  :", RESULTS_DIR)
print("DISAGREE_CSV :", DISAGREE_CSV)

# Load environment variables (expects .env with OPENAI_API_KEY)
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", ""))


PROJECT_ROOT : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_비밀게시판
RESULTS_DIR  : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_비밀게시판\results
DISAGREE_CSV : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_비밀게시판\results\disagreement_cases_with_text.csv


In [2]:
# 1) 사람 피드백용 템플릿 만들기

if not DISAGREE_CSV.exists():
    raise FileNotFoundError(f"{DISAGREE_CSV} 가 없습니다. 먼저 분석 노트북을 돌려 disagreement_cases_with_text.csv를 생성하세요.")

df_dis = pd.read_csv(DISAGREE_CSV)

print("원본 disagreement_cases_with_text.csv 예시:")
display(df_dis.head())

# human_preferred_decision 컬럼 추가 (빈 값)
if "human_preferred_decision" not in df_dis.columns:
    df_dis["human_preferred_decision"] = ""

# 설명용 컬럼도 하나 추가할 수 있음 (선택)
if "human_comment" not in df_dis.columns:
    df_dis["human_comment"] = ""

TEMPLATE_PATH = RESULTS_DIR / "disagreement_with_human_template.csv"
df_dis.to_csv(TEMPLATE_PATH, index=False, encoding="utf-8-sig")

print("\n사람 라벨링 템플릿을 저장했습니다:")
print(" ->", TEMPLATE_PATH)
print("\n이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:")
print(" - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나")
print(" - human_comment (선택): 왜 그렇게 판단했는지 간단 설명")


원본 disagreement_cases_with_text.csv 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text
0,4,BLOCK,BLOCK,WARN_AND_ALLOW,2,post,0,어효 코스프레하는 애들은 하나같이 젖크고 얼굴 이쁘고 빼빼 말랐네
1,14,BLOCK,WARN_AND_ALLOW,ALLOW,4,post,0,과제가 공부보다 훨씬 덜 고통스럽다고 느끼는 건 아직 저학년이라서 그런 건가? 공부...
2,17,BLOCK,ALLOW,ALLOW,4,comment,3,"비슷한 거 느껴본적 있는데 년차 쌓이니까 뭐가 문제인지 깨달음. ppt만 보지말고,..."
3,34,BLOCK,WARN_AND_ALLOW,ALLOW,7,post,0,ㅈㅈㅣ팩트 일정 이상의 길이부터는 두께 강직도 지속력이 더 중요하다 반박시 섹알못
4,35,BLOCK,WARN_AND_ALLOW,ALLOW,7,comment,1,그니까 너가 딱 평균이란 거지?



사람 라벨링 템플릿을 저장했습니다:
 -> C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_비밀게시판\results\disagreement_with_human_template.csv

이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:
 - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나
 - human_comment (선택): 왜 그렇게 판단했는지 간단 설명


## 2) 사람이 할 일

1. 방금 생성된 `disagreement_with_human_template.csv` 파일을 엽니다.
2. 각 row에 대해:
   - **human_preferred_decision**: `BLOCK`, `WARN_AND_ALLOW`, `ALLOW` 중 하나를 입력
   - **human_comment**: (선택) "자기비하라서 허용", "성기 언급은 최소 경고" 등 이유를 적어도 좋음
3. 수정한 파일을 예를 들어 `disagreement_with_human_filled.csv` 같은 이름으로 저장합니다.
   - 템플릿 파일을 그대로 덮어써도 되지만,
   - 원본을 남기고 싶으면 새 파일 이름을 쓰세요.

아래 셀에서는, 사람이 채운 CSV 파일 경로를 `FILLED_PATH` 변수로 지정해서 사용합니다.


In [3]:
# 3) 사람이 채운 피드백 파일 불러오기

# 사람이 채운 파일 이름을 여기서 지정하세요.
FILLED_PATH = RESULTS_DIR / "disagreement_with_human_filled.csv"  # 예시 이름

if not FILLED_PATH.exists():
    raise FileNotFoundError(
        f"{FILLED_PATH} 가 없습니다. disagreement_with_human_template.csv를 채운 뒤, "
        "이름을 disagreement_with_human_filled.csv 로 저장하거나, 위 경로를 원하는 이름으로 바꾸세요."
    )

df_feedback = pd.read_csv(FILLED_PATH)

print("사람 피드백이 포함된 데이터 예시:")
display(df_feedback.head())

# human_preferred_decision이 비어있는 row는 제외 (필요시)
df_feedback = df_feedback[df_feedback["human_preferred_decision"].notna() & (df_feedback["human_preferred_decision"] != "")]
print(f"유효한 피드백 row 수: {len(df_feedback)}")


사람 피드백이 포함된 데이터 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text,human_preferred_decision,human_comment
4,BLOCK,BLOCK,WARN_AND_ALLOW,2,post,0,어효 코스프레하는 애들은 하나같이 젖크고 얼굴 이쁘고 빼빼 말랐네,BLOCK,BLOCK,NaN
14,BLOCK,WARN_AND_ALLOW,ALLOW,4,post,0,과제가 공부보다 훨씬 덜 고통스럽다고 느끼는 건 아직 저학년이라서 그런 건가? 공부...,ALLOW,ALLOW,NaN
17,BLOCK,ALLOW,ALLOW,4,comment,3,"비슷한 거 느껴본적 있는데 년차 쌓이니까 뭐가 문제인지 깨달음. ppt만 보지말고,...",ALLOW,ALLOW,NaN
34,BLOCK,WARN_AND_ALLOW,ALLOW,7,post,0,ㅈㅈㅣ팩트 일정 이상의 길이부터는 두께 강직도 지속력이 더 중요하다 반박시 섹알못,BLOCK,BLOCK,NaN
35,BLOCK,WARN_AND_ALLOW,ALLOW,7,comment,1,그니까 너가 딱 평균이란 거지?,WARN_AND_ALLOW,WARN_AND_ALLOW,NaN


유효한 피드백 row 수: 25


In [4]:
# === NEW: 사람 라벨과 정책 A/B/C 일치율 계산 ============================
policy_cols = ["A", "B", "C"]
stats = []

for p in policy_cols:
    matches = (df_feedback[p] == df_feedback["human_preferred_decision"]).sum()
    total = len(df_feedback)
    acc = matches / total if total > 0 else float("nan")
    stats.append((p, matches, total, acc))

df_stats = pd.DataFrame(stats, columns=["policy", "n_match", "n_total", "accuracy"])

print("\n=== Human–policy agreement summary ===")
print(df_stats)

# 가장 사람과 비슷한 정책 선택
best_row = df_stats.sort_values("accuracy", ascending=False).iloc[0]
recommended_policy = best_row["policy"]

print(f"\n현재 사람 라벨과 가장 가까운 정책: Policy {recommended_policy} "
      f"(정확도 {best_row['accuracy']:.3f})")

# LLM 프롬프트에 넣을 수 있도록 텍스트 블록으로 변환
stats_text_lines = []
for _, row in df_stats.iterrows():
    stats_text_lines.append(
        f"- Policy {row['policy']}: "
        f"matches = {row['n_match']} / {int(row['n_total'])}, "
        f"accuracy = {row['accuracy']:.3f}"
    )
stats_block = "\n".join(stats_text_lines)
# ======================================================



=== Human–policy agreement summary ===
  policy  n_match  n_total  accuracy
0      A       10       25      0.40
1      B       14       25      0.56
2      C        0       25      0.00

현재 사람 라벨과 가장 가까운 정책: Policy B (정확도 0.560)


## 4) 기존 Policy A/B/C v1 텍스트 입력

아래 셀의 `POLICY_A_V1`, `POLICY_B_V1`, `POLICY_C_V1` 문자열 안에  
**지금 시뮬레이션에 사용하고 있는 정책 설명(프롬프트용 규칙 텍스트)**를 그대로 붙여 넣으면 됩니다.

- 너무 길어도 괜찮지만, 가능하면 **사람이 읽을 수 있을 정도로 정리된 버전**이 좋습니다.
- 나중에 LLM에게는 이 텍스트와 `df_feedback`를 함께 넘겨서,
  - 정책이 어디서 과도하게 차단/허용하는지
  - 사람 피드백과 더 잘 맞도록 어떻게 수정하면 좋은지
  제안하도록 요청합니다.


In [5]:
# TODO: 여기에 현재 사용중인 A/B/C 정책 설명을 붙여 넣으세요.

POLICY_A_V1 = """Policy A: High Protection, Low Tolerance (매너 최우선)

- BLOCK if:
  - Any direct insult or derogatory expression toward a person or group
    (예: "멍청이", "병x", "저년", "틀딱" 등).
  - Swearing or profanity directed at a person, group, or their work
    (예: "씨발", "좆같은 과제", "저 교수 또 ㅈ같이 하네").
  - Sarcastic or mocking tone that targets a person’s character, ability, or identity.
  - Shaming of appearance, background, academic ability, or mental health.
  - Any hate speech or slur toward social groups (gender, region, race, sexuality, etc.).
  - Any threat, doxxing, or suggestion of harm.

- WARN_AND_ALLOW if:
  - Strong criticism of ideas, policies, or institutions without personal insult.
  - Rude words aimed at a system or policy, not at a person.

- ALLOW if:
  - Polite or neutral comments.
  - Constructive criticism with respectful language.
  - Jokes or slang that are not targeted and not hateful.

Principle: When in doubt, prioritize emotional safety and manner
even if some harsh but meaningful criticism is lost.
"""

POLICY_B_V1 = """Policy B: Balanced (매너와 토론의 균형)

- BLOCK if:
  - Direct, explicit personal attack to a specific individual using insults or slurs.
  - Hate speech toward a protected/vulnerable group (gender, sexuality, race, disability, region, etc.).
  - Threats of violence, doxxing, or calls for harassment.

- WARN_AND_ALLOW if:
  - Strong, harsh expressions that mainly target ideas, policies, classes, or systems.
  - Profanity is present but not clearly directed at a specific person.
  - Mild sarcasm but focused on a public issue (tuition, dorm policy, grading, etc.).

- ALLOW if:
  - Criticism is expressed in relatively respectful language.
  - Comment contributes arguments, experiences, or information.
  - Emotional expressions that are not direct harassment or hate speech.

Principle: When in doubt, preserve debate value if the target is
an idea or policy, but never allow direct harassment or hate speech.
"""

POLICY_C_V1 = """Policy C: Minimal Regulation, High Freedom (최소 규제, 표현의 자유 우선)

- BLOCK if:
  - Severe insult or slur naming a person or clearly identifiable small group,
    combined with clear hostility or harm.
  - Clear hate speech toward protected groups using abusive slurs.
  - Explicit threats of violence, doxxing, or incitement to harassment.

- WARN_AND_ALLOW if:
  - Harsh, emotional language with insults or profanity that is not an explicit threat
    and not targeted at a protected group.
  - Rude mockery that is uncomfortable but not clearly hateful or threatening.

- ALLOW if:
  - Almost any criticism, even if rude, as long as:
    - No explicit threats.
    - No hate speech.
    - No doxxing.

Principle: When in doubt, err on the side of freedom of expression,
and only block content that is clearly hateful or threatening.
"""

print("Policy A v1 길이:", len(POLICY_A_V1))
print("Policy B v1 길이:", len(POLICY_B_V1))
print("Policy C v1 길이:", len(POLICY_C_V1))

Policy A v1 길이: 1022
Policy B v1 길이: 909
Policy C v1 길이: 831


In [6]:
# 5) LLM에 넘길 프롬프트 텍스트 구성

def build_feedback_snippet(df: pd.DataFrame, max_rows: int = 20) -> str:
    """사람 피드백 테이블을 간단한 텍스트 블록으로 변환"""
    rows = []
    subset = df.head(max_rows)
    for _, r in subset.iterrows():
        row_txt = textwrap.dedent(f"""        - sample_id: {r['sample_id']}
          role: {r['role']}, thread_id: {r['thread_id']}
          original_text: {r['text']}
          policy_decisions: A={r['A']}, B={r['B']}, C={r['C']}
          human_preferred_decision: {r['human_preferred_decision']}
          human_comment: {r.get('human_comment', '')}
        """)
        rows.append(row_txt)
    return "\n".join(rows)


feedback_block = build_feedback_snippet(df_feedback, max_rows=30)

system_prompt = """You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v1).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v2) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v2', 'Policy B v2', 'Policy C v2'.
- For each policy, include:
  - A 3~6 bullet outline of the main principles.
  - Clear guidance on when to choose BLOCK / WARN_AND_ALLOW / ALLOW.
  - If helpful, 2~3 short example patterns of language that should be BLOCKed vs ALLOWed.
- At the end, add a short subsection 'Final recommendation' where you:
  - Recommend which policy (A, B, or C) should be the default 'final' policy after this iteration,
  - Justify your recommendation in 2–4 sentences, using both the stats and qualitative patterns.
"""

user_prompt = f"""## Current policies (v1)

### Policy A v1
{POLICY_A_V1}

### Policy B v1
{POLICY_B_V1}

### Policy C v1
{POLICY_C_V1}

---

## Human feedback on disagreement cases

Below are some cases where policy A/B/C disagreed, and humans provided their preferred decisions.

{feedback_block}

---

## Aggregate agreement stats (human vs policy decisions)

The following are aggregate agreement rates between each policy's decisions (A/B/C) 
and the human_preferred_decision on the feedback dataset:

{stats_block}

---

Please propose refined versions 'Policy A v2', 'Policy B v2', 'Policy C v2' as described in the system instructions.
Also, based on the agreement stats and qualitative patterns, provide a short 'Final recommendation'
section indicating which policy (A, B, or C) is the best candidate to adopt as the final policy after this iteration, and why.
"""


print("=== System prompt preview ===\n")
print(system_prompt)

print("=== User prompt preview ===\n")
print(user_prompt)


=== System prompt preview ===

You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v1).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v2) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v2', 'Policy B v2', 'Policy C v2'.
- For each policy, include:
  - A 3~6 bullet outline 

In [7]:
# 6) LLM 호출 (정책 v2 제안 받기)

response = client.chat.completions.create(
    model="gpt-5.1",  # 필요에 따라 다른 모델로 변경 가능
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.3,
)

answer = response.choices[0].message.content
print(answer)

# 결과를 파일로 저장
POLICY_V2_PATH = RESULTS_DIR / "policy_v2_suggestions.md"
with open(POLICY_V2_PATH, "w", encoding="utf-8") as f:
    f.write(answer)

print("\n정책 v2 제안 내용을 저장했습니다:")
print(" ->", POLICY_V2_PATH)


### Policy A v2 – High Protection, Low Tolerance (매너 최우선)

**Main principles**

- Strongly prioritize emotional safety over freedom of expression.
- Block most content that includes insults, mocking, or aggressive tone toward people.
- Treat ambiguous or borderline cases as at least WARN_AND_ALLOW, not ALLOW.
- Allow only clearly polite, neutral, or gently critical comments.

**When to BLOCK**

Block if **any** of the following are present:

- Direct insult, derogatory term, or curse toward:
  - a person, group, or their work/effort  
  - 예: “멍청이”, “병x”, “저년”, “틀딱”, “씨발 교수”, “좆같은 과제”.
- Swearing or profanity **clearly tied to** a person, group, or their work:
  - 예: “저 교수 또 ㅈ같이 하네”, “이 수업 진짜 ㅈ같이 만들어놨네”.
- Sarcastic or mocking tone that targets character, ability, or identity:
  - 예: “저 정도 머리로 대학은 어떻게 왔냐?”, “그 정도면 그냥 포기해라”.
- Shaming of appearance, background, academic ability, or mental health.
- Any hate speech or slur toward social groups (gender, region, race, sexuality, etc.).
- An